## models
- 模型：包含要存储的数据的基本字段和行为，通常，每个模型都映射到单个数据库表
    - 每个模型都是django.db.model.Model的子类
    - 模型的每个属性代表一个数据库字段

In [ ]:
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)
    
#上面的内容会按照下面的方式在数据库创建
CREATE TABLE myapp_person (
    "id" serial NOT NULL PRIMARY KEY,
    "first_name" varchar(30) NOT NULL,
    "last_name" varchar(30) NOT NULL
);

## 模型的组成

### 字段
- 模型唯一需要的部分
- 第一的数据库字段列表
- 字段由类属性指定

In [ ]:
from django.db import models

class Musician(models.Model):
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    instrument = models.CharField(max_length=100)

#### 字段类型
- 每个字段都应该是对应Field类的实例


#### 字段选项 -- 常用
- null：如果为true，django将以null在数据库中存储空值，默认为false
- blank：如果为true，该字段允许为空，默认为false，如果为false代表必填
- choices：2元组的可迭代，用作字段的选项，默认以选择框显示
    - 第一个元素是将存储在数据库中的值
    - 第二个元素是显示在选择框中的值
- default：字段的默认值，可以是值或可调用对象，如果可调用，则每次创建新对象都会调用
- help_text：使用表单小部件显示的额外"帮助"文本
- primary_key：如果True，则为主键，如果没有设置，则将自动创建一个IntegerField来保存主键        
        id = models.AutoField(primary_key=True)
- unique：如果为true，该字段在整个表格中必须是唯一的
- verbose_name：字段的详细名称，如果不给出，Django会使用字段的属性名称自动创建
- 

In [ ]:
# choices
YEAR_IN_SCHOOL_CHOICES = (
    ('FR', 'Freshman'),
    ('SO', 'Sophomore'),
    ('JR', 'Junior'),
    ('SR', 'Senior'),
    ('GR', 'Graduate'),
)

### 关系

#### 多对一关系
- 使用ForeginKey

In [ ]:
from django.db import models

class Manufacturer(models.Model):
    # ...
    pass

class Car(models.Model):
    manufacturer = models.ForeignKey(Manufacturer, on_delete=models.CASCADE)
    # ...

#### 多对多关系
- 使用ManyToManyField

In [ ]:
from django.db import models

class Topping(models.Model):
    # ...
    pass

class Pizza(models.Model):
    # ...
    toppings = models.ManyToManyField(Topping)

## models的api

### 创建

In [ ]:
# 创建
from blog.models import Blog
b = Blog(name='Beatles Blog', tagline='All the latest Beatles news.')
b.save()

### 更新

In [ ]:
# 更新
b5.name = 'New name'
b5.save()

#### 更新ForeignKey字段

In [ ]:
# 保存ForeignKey字段
from blog.models import Blog, Entry
entry = Entry.objects.get(pk=1)
cheese_blog = Blog.objects.get(name="Cheddar Talk")
entry.blog = cheese_blog
entry.save()

#### 更新ManyToManyField字段

In [ ]:
from blog.models import Author
joe = Author.objects.create(name="Joe")
entry.authors.add(joe)

# 增加多个
john = Author.objects.create(name="John")
paul = Author.objects.create(name="Paul")
entry.authors.add(john, paul, george, ringo)

### 检索

#### 检索所有对象

In [ ]:
all_entries = Entry.objects.all()

#### 使用过滤器
- filter(\*\*kwargs)：返回QuerySet包含与给定查找参数匹配的新对象
- exclude(\*\*kwargs)：返回QuerySet包含与给定查找参数不匹配的新对象

In [ ]:
Entry.objects.filter(pub_date__year=2006)

#### 检索单个对象

In [ ]:
one_entry = Entry.objects.get(pk=1) # 找不到会返回DoesNotExist 异常
# 如果多个项目与get匹配，则会返回DoesNotExist 

### 排序

In [ ]:
Entry.objects.order_by('headline')

### 字段查找

In [ ]:
# 
Entry.objects.filter(pub_date__lte='2006-01-01')
# 等同于
SELECT * FROM blog_entry WHERE pub_date <= '2006-01-01';

# “精确”匹配
Entry.objects.get(headline__exact="Cat bites dog")
# 等同于
SELECT ... WHERE headline = 'Cat bites dog';

# iexact 不区分大小写的匹配项
Blog.objects.get(name__iexact="beatles blog")

# contains：包含
Entry.objects.get(headline__contains='Lennon')
# 类似于
SELECT ... WHERE headline LIKE '%Lennon%';

### 复杂查找 -- Q

In [ ]:
# 或
Q(question__startswith='Who') | Q(question__startswith='What')
# 等同于
WHERE question LIKE 'Who%' OR question LIKE 'What%'

# 否
Q(question__startswith='Who') | ~Q(pub_date__year=2005)
# 等同于
WHERE question LIKE 'Who%' OR pub_date.year!=2005

# 并
Poll.objects.get(
    Q(question__startswith='Who'),
    Q(pub_date=date(2005, 5, 2)) | Q(pub_date=date(2005, 5, 6))
)
# 等同于
SELECT * from polls WHERE question LIKE 'Who%' AND (pub_date = '2005-05-02' OR pub_date = '2005-05-06')



### 删除对象
- 立即删除对象并返回已删除的对象数和具有每个对象类型的删除数的字典

In [ ]:
e.delete()